# Boilerplate

There's probably a better way than what follows

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

What can we run?

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__')]

['EndpointW',
 'PacketW',
 'RequestActionW',
 'TransportInterface',
 'nwv2_python_wrapper']

Run it

In [4]:
tiface = TransportInterface("0.0.0.0", 12345)

thread '<unnamed>' panicked at 'there is no reactor running, must be called from the context of a Tokio 1.x runtime', /home/aaron/.cargo/registry/src/github.com-1ecc6299db9ec823/tokio-1.16.1/src/runtime/context.rs:29:26
stack backtrace:
   0: rust_begin_unwind
             at /rustc/a8314ef7d0ec7b75c336af2c9857bfaf43002bfc/library/std/src/panicking.rs:584:5
   1: core::panicking::panic_fmt
             at /rustc/a8314ef7d0ec7b75c336af2c9857bfaf43002bfc/library/core/src/panicking.rs:142:14
   2: core::panicking::panic_display
             at /rustc/a8314ef7d0ec7b75c336af2c9857bfaf43002bfc/library/core/src/panicking.rs:72:5
   3: core::panicking::panic_str
             at /rustc/a8314ef7d0ec7b75c336af2c9857bfaf43002bfc/library/core/src/panicking.rs:56:5
   4: core::option::expect_failed
             at /rustc/a8314ef7d0ec7b75c336af2c9857bfaf43002bfc/library/core/src/option.rs:1854:5
   5: core::option::Option<T>::expect
             at /rustc/a8314ef7d0ec7b75c336af2c9857bfaf43002bfc/libr

PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime